#A.7 用Numba编写快速NumPy函数

Numba是一个开源项目，它可以利用CPUs、GPUs或其它硬件为类似NumPy的数据创建快速函数。它使用了LLVM项目（http://llvm.org/），将Python代码转换为机器代码。

为了介绍Numba，来考虑一个纯粹的Python函数，它使用for循环计算表达式(x - y).mean()：

In [1]:
import numpy as np

In [2]:

def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0

    for i in range(nx):
        result += x[i] - y[i]
        count += 1

    return result / count

这个函数很慢：

In [7]:
x = np.random.randn(10000000)
y = np.random.randn(10000000)

In [4]:
%timeit mean_distance(x, y)

1.19 s ± 31.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit (x - y).mean()

17.2 ms ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


NumPy的版本要比它快过100倍。我们可以转换这个函数为编译的Numba函数，使用numba.jit函数：

In [3]:
import numba as nb

In [4]:
numba_mean_distance = nb.jit(mean_distance)

也可以写成装饰器：

In [5]:
@nb.jit
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

它要比矢量化的NumPy快：

In [9]:
%timeit numba_mean_distance(x, y)

6 ms ± 35.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Numba不能编译Python代码，但它支持纯Python写的一个部分，可以编写数值算法。

Numba是一个深厚的库，支持多种硬件、编译模式和用户插件。它还可以编译NumPy Python API的一部分，而不用for循环。Numba也可以识别可以便以为机器编码的结构体，但是若调用CPython API，它就不知道如何编译。Numba的jit函数有一个选项，nopython=True，它限制了可以被转换为Python代码的代码，这些代码可以编译为LLVM，但没有任何Python C API调用。jit(nopython=True)有一个简短的别名numba.njit。

前面的例子，我们还可以这样写：

In [10]:
from numba import float64, njit


@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

我建议你学习Numba的线上文档（http://numba.pydata.org/）。下一节介绍一个创建自定义Numpy ufunc对象的例子。

## 用Numba创建自定义numpy.ufunc对象

numba.vectorize创建了一个编译的NumPy ufunc，它与内置的ufunc很像。考虑一个numpy.add的Python例子：

In [12]:
from numba import vectorize


@vectorize
def nb_add(x, y):
    return x + y

现在有：

In [14]:
x = np.arange(10)

nb_add(x, x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [15]:
nb_add.accumulate(x, 0)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])